<center><h1>ARIMA TRAINING</h1><center>

In [23]:
!pip install pmdarima


  Using cached pmdarima-2.0.4.tar.gz (630 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pmdarima


  error: subprocess-exited-with-error
  
  × Building wheel for pmdarima (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [43 lines of output]
      Partial import of pmdarima during the build process.
      
      Requirements: ['joblib>=0.11\nCython>=0.29,!=0.29.18,!=0.29.31\nnumpy>=1.21.2\npandas>=0.19\nscikit-learn>=0.22\nscipy>=1.3.2\nstatsmodels>=0.13.2\nurllib3\nsetuptools>=38.6.0,!=50.0.0\npackaging>=17.1  # Bundled with setuptools, but want to be explicit\n']
      Adding extra setuptools args
      Setting up with setuptools
      Traceback (most recent call last):
        File "C:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Users\jonaz\AppData\Local\Programs\Python\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def forecast_arima_by_genre(df, genres):
    """
    Forecasts genre popularity using ARIMA and evaluates accuracy.

    Parameters:
        df (pd.DataFrame): The input DataFrame with columns:
            - 'playlist_genre'
            - 'track_album_release_date'
            - 'median_track_popularity'
        genres (list): List of genre strings to process.
    """

    for genre in genres:
        print(f"\n=== ARIMA Forecast for: {genre.upper()} ===")

        # Filter and prepare time series
        df_genre = df[df['playlist_genre'] == genre].sort_values('track_album_release_date')
        ts = df_genre.set_index('track_album_release_date')['median_track_popularity']

        # Ensure time index is PeriodIndex
        ts.index = pd.PeriodIndex(ts.index, freq='Y')

        # ADF test for stationarity
        adf_result = adfuller(ts)
        print(f"ADF p-value: {adf_result[1]:.4f}")
        if adf_result[1] > 0.05:
            print(" → Non-stationary, applying differencing.")
            ts_diff = ts.diff().dropna()
        else:
            ts_diff = ts

        # Fit ARIMA(1,1,1)
        model = ARIMA(ts, order=(1, 1, 1))
        model_fit = model.fit()

        # Forecast for next 5 years
        forecast = model_fit.forecast(steps=5)
        forecast_index = pd.period_range(start=ts.index.max() + 1, periods=5, freq='Y').to_timestamp()
        print("Forecast (2025–2029):")
        print(forecast)

        # Plot
        plt.figure(figsize=(8, 4))
        plt.plot(ts.index.to_timestamp(), ts.values, label='Actual')
        plt.plot(forecast_index, forecast.values, label='Forecast', linestyle='--')
        plt.title(f"ARIMA Forecast for '{genre}' (2025–2029)")
        plt.xlabel("Year")
        plt.ylabel("Median Track Popularity")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(f"{genre}_arima_forecast.png")  # Save plot to file
        plt.show()
        print(f"→ Saved plot as '{genre}_arima_forecast.png'")

        # Accuracy check on last 2 years
        if ts.index.max().year >= 2024:
            y_true = ts[-2:]
            y_pred = model_fit.predict(start=y_true.index[0], end=y_true.index[-1])
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            print(f"MAE: {mae:.2f} | RMSE: {rmse:.2f}")

In [ ]:
df = pd.read_csv("05_genres_median_audio_features.csv")
genres = ['pop', 'hip-hop', 'latin', 'electronic']

forecast_arima_by_genre(df, genres)
